In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [ ]:
zipurl = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip'

with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('/tmp/Bike-Sharing-Dataset')


In [ ]:
bikes_hour_df_raw=pd.read_csv('/tmp/Bike-Sharing-Dataset/hour.csv')
bikes_day_df_raw = pd.read_csv('/tmp/Bike-Sharing-Dataset/day.csv')

In [ ]:
bikes_hour_df_raw.describe()

In [ ]:
bikes_day_df_raw.describe()

In [ ]:
bikes_hour_df_raw.head()

In [ ]:
bikes_hour_df = bikes_hour_df_raw.drop(['casual','registered'],axis=1)

In [ ]:
bikes_hour_df['cnt'].describe()

In [ ]:
fig,ax = plt.subplots(1)
ax.plot(sorted(bikes_hour_df['cnt']),color='orange')
ax.set_xlabel("Row index",fontsize=12)
ax.set_ylabel("Sorted Renntal Counts",fontsize=12)
fig.suptitle("Outpcome Variable -cnt -Rental Counts'")
plt.show()

In [ ]:
counter_=0
for tp in bikes_hour_df.dtypes:
    if(tp=="float64"):
        bikes_hour_df = bikes_hour_df.sort_values(bikes_hour_df.dtypes.index[counter_])
        feature = bikes_hour_df.dtypes.index[counter_]
        plt.scatter(bikes_hour_df[feature],bike_hour_df['cnt'])
        plt.suptitle("Numerical Feature: Cnt VS" + feature)
        plt.xlabel(feature)
        plt.ylabel("Count of All Bikes Rented")
        plt.show()
    counter_ +=1

NameError: name 'bikes_hour_df' is not defined